## Asian Restaurant in Tornoto

In [1]:
#install Beautifulsoup4 and other packages
!conda install -c conda-forge beautifulsoup4 --yes
!conda install -c conda-forge lxml --yes
!conda install -c conda-forge html5lib --yes
!conda install -c conda-forge geocoder --yes
!conda install -c conda-forge folium=0.5.0 --yes

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following packages will be UPDATED:

    beautifulsoup4: 4.6.0-py35h442a8c9_1 --> 4.6.3-py35_0 conda-forge

beautifulsoup4 100% |################################| Time: 0:00:00  39.43 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following packages will be UPDATED:

    libxml2: 2.9.4-h6b072ca_5     --> 2.9.8-h422b904_2     conda-forge
    libxslt: 1.1.29-hcf9102b_5    --> 1.1.32-h88dbc4e_2    conda-forge
    lxml:    4.1.0-py35ha401a81_0 --> 4.2.5-py35hc9114bc_0 conda-forge

libxml2-2.9.8- 100% |################################| Time: 0:00:00  73.79 MB/s
libxslt-1.1.32 100% |################################| Time: 0:00:00  65.88 MB/s
lxml-4.2.5-py3 100% |################################| Time: 0:00:00  67.33 MB/s
Fetchin

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.geocoders import Nominatim
import matplotlib.pyplot as plt

## Get the data from wiki page

In [35]:
source=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(source, 'lxml')

In [36]:
#read all neighourhood of Toronto from xml and load into dataframe
table=soup.find('table', attrs={'class':'wikitable sortable'})
table_rows=table.find_all('tr')
res=[]
i=0
for tr in table_rows:
     td=tr.find_all('td')
     row=[tr.text.strip() for tr in td if tr.text.strip()]
     if row:
            res.append(row)

df=pd.DataFrame(res,columns=["Postcode","Borough","Neighbourhood"])
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [37]:
#Remove inadequate data
df1=df[df['Borough']!= 'Not assigned']
print("Before: ", df1[df1['Neighbourhood'] == 'Not assigned'].shape)
df1['Neighbourhood'].replace(regex='Not assigned', value=df1['Borough'], inplace=True)
print("After: ", df1[df1['Neighbourhood'] == 'Not assigned'].shape)

Before:  (1, 3)
After:  (0, 3)


/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/pandas/core/generic.py:4619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [38]:
#group all neighbourhood by postcode
df2=df1.groupby(['Postcode','Borough'])['Neighbourhood'].agg(lambda x: ', '.join(x))
df2=pd.DataFrame(df2.reset_index())
df2.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [39]:
#download and read lat & lng of Troronto postal code data
!wget -q -O 'Geospatial_Coordinates.csv' http://cocl.us/Geospatial_data

In [40]:
latlng=pd.read_csv("Geospatial_Coordinates.csv")
latlng.columns=["Postcode", "Latitude", "Longitude"]
print ('Neighbourhood df: ', df2.shape)
print ('post code lat n lng df: ', latlng.shape)

Neighbourhood df:  (103, 3)
post code lat n lng df:  (103, 3)


In [41]:
#join two dataframes
df3=pd.merge(df2, latlng, on='Postcode')
df3.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Cluster neighbourhood in Toronto

In [42]:
address = 'Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Lat n lng of Toronto city are {},{}'.format(latitude, longitude))

Lat n lng of Toronto city are 43.653963,-79.387207


In [43]:
# create map using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df3['Latitude'], df3['Longitude'], df3['Borough'], df3['Neighbourhood']):
    label='{}, {}'.format(borough, neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
#        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Exploring only neighborhoods in the burough of Central Toronto

In [44]:
central_toronto = df3[df3['Borough']=='Central Toronto'].reset_index(drop=True)
central_toronto

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,M4P,Central Toronto,Davisville North,43.712751,-79.390197
2,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
3,M4S,Central Toronto,Davisville,43.704324,-79.388790
4,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
5,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049
6,M5N,Central Toronto,Roselawn,43.711695,-79.416936
7,M5P,Central Toronto,"Forest Hill North, Forest Hill West",43.696948,-79.411307
8,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678


## Define Foursquare credentials and version

In [45]:
CLIENT_ID = 'VKVGWTSR42A1AG01PGXZGR1XOYTX3DDYJHTCQDL4ZA003V25' # your Foursquare ID
CLIENT_SECRET = 'JWSZVEEYZBFADYMMNEZCSHBCG5KD2B1PXL4KTXFKAVZ3Z34F' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
radius = 2000 #set radius parameter to be searched

In [46]:
def getNearbyLaundryVenues(names, latitudes, longitudes):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng,
            'Asian Restaurant',
            radius, 
            100)
            
        # make the GET request
        results = requests.get(url).json()["response"]['venues']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['name'], 
            v['location']['lat'], 
            v['location']['lng'],  
            v['categories'][0]['name']) for v in results if v['categories']])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [47]:
ind_res = getNearbyLaundryVenues(central_toronto['Neighbourhood'],central_toronto["Latitude"],central_toronto['Longitude'])
ind_res.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,43.72802,-79.38879,Lime Asian Kitchen,43.728995,-79.403047,Asian Restaurant
1,Lawrence Park,43.72802,-79.38879,Chi Asian Tapas,43.710688,-79.399181,Asian Restaurant
2,Lawrence Park,43.72802,-79.38879,Herbs Restaurant,43.726178,-79.399680,Bar
3,Lawrence Park,43.72802,-79.38879,Gamberoni,43.729515,-79.403477,Italian Restaurant
4,Lawrence Park,43.72802,-79.38879,Phayao Thai Restaurant,43.733720,-79.404335,Thai Restaurant


In [48]:
ind_res.groupby('Neighborhood').count().head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Davisville,41,41,41,41,41,41
Davisville North,38,38,38,38,38,38
"Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West",46,46,46,46,46,46
"Forest Hill North, Forest Hill West",45,45,45,45,45,45
Lawrence Park,26,26,26,26,26,26


In [49]:
# One hot encoding

ind_res_hot_code = pd.get_dummies(ind_res[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ind_res_hot_code['Neighborhood'] = ind_res['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [ind_res_hot_code.columns[-1]] + list(ind_res_hot_code.columns[:-1])
ind_res_hot_code = ind_res_hot_code[fixed_columns]
ind_res_hot_code.head()

,Neighborhood,African Restaurant,Asian Restaurant,BBQ Joint,Bar,Bistro,Breakfast Spot,Café,Cantonese Restaurant,Caribbean Restaurant,...,Office,Pub,Restaurant,Sandwich Place,Steakhouse,Sushi Restaurant,Tapas Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,Lawrence Park,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Lawrence Park,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Lawrence Park,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Lawrence Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Lawrence Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [50]:
ind_res_grouped = ind_res_hot_code.groupby('Neighborhood').mean().reset_index()
ind_res_grouped.head()

,Neighborhood,African Restaurant,Asian Restaurant,BBQ Joint,Bar,Bistro,Breakfast Spot,Café,Cantonese Restaurant,Caribbean Restaurant,...,Office,Pub,Restaurant,Sandwich Place,Steakhouse,Sushi Restaurant,Tapas Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,Davisville,0.0,0.048780,0.000000,0.000000,0.000000,0.048780,0.000000,0.024390,0.0,...,0.024390,0.024390,0.121951,0.024390,0.048780,0.048780,0.024390,0.097561,0.000000,0.000000
1,Davisville North,0.0,0.078947,0.026316,0.026316,0.000000,0.052632,0.000000,0.000000,0.0,...,0.026316,0.026316,0.131579,0.026316,0.052632,0.052632,0.000000,0.105263,0.000000,0.000000
2,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",0.0,0.021739,0.000000,0.000000,0.021739,0.000000,0.021739,0.021739,0.0,...,0.000000,0.000000,0.152174,0.065217,0.000000,0.021739,0.000000,0.021739,0.021739,0.021739
3,"Forest Hill North, Forest Hill West",0.0,0.088889,0.000000,0.000000,0.000000,0.022222,0.022222,0.022222,0.0,...,0.022222,0.000000,0.111111,0.022222,0.044444,0.022222,0.022222,0.088889,0.000000,0.022222
4,Lawrence Park,0.0,0.115385,0.038462,0.038462,0.000000,0.076923,0.000000,0.000000,0.0,...,0.038462,0.000000,0.153846,0.038462,0.038462,0.000000,0.000000,0.153846,0.000000,0.000000


In [51]:
num_top_venues = 5
for hood in ind_res_grouped['Neighborhood'][:3]:
    print("----"+hood+"----")
    temp =ind_res_grouped[ind_res_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Davisville----
                 venue  freq
0           Restaurant  0.12
1      Thai Restaurant  0.10
2   Italian Restaurant  0.10
3    Indian Restaurant  0.05
4  Japanese Restaurant  0.05


----Davisville North----
                venue  freq
0          Restaurant  0.13
1     Thai Restaurant  0.11
2  Italian Restaurant  0.08
3    Asian Restaurant  0.08
4      Breakfast Spot  0.05


----Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West----
                 venue  freq
0           Restaurant  0.15
1   Chinese Restaurant  0.09
2   Italian Restaurant  0.09
3       Sandwich Place  0.07
4  Japanese Restaurant  0.07




In [52]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [53]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = ind_res_grouped['Neighborhood']

for ind in np.arange(ind_res_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ind_res_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Davisville,Restaurant,Thai Restaurant,Italian Restaurant,Indian Restaurant,Sushi Restaurant,Steakhouse,Japanese Restaurant,Asian Restaurant,Diner,Breakfast Spot
1,Davisville North,Restaurant,Thai Restaurant,Asian Restaurant,Italian Restaurant,Sushi Restaurant,Steakhouse,Diner,Breakfast Spot,Indian Restaurant,Chinese Restaurant
2,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",Restaurant,Italian Restaurant,Chinese Restaurant,Indian Restaurant,Sandwich Place,Japanese Restaurant,Hotel,Cantonese Restaurant,Café,Bistro
3,"Forest Hill North, Forest Hill West",Restaurant,Thai Restaurant,Asian Restaurant,Italian Restaurant,Diner,Japanese Restaurant,Chinese Restaurant,Steakhouse,General Entertainment,French Restaurant
4,Lawrence Park,Thai Restaurant,Restaurant,Asian Restaurant,Italian Restaurant,Diner,Breakfast Spot,Jazz Club,Churrascaria,Japanese Restaurant,Office


In [54]:
kclusters = 4

ind_res_grouped_clustering = ind_res_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ind_res_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([2, 2, 1, 2, 0, 1, 0, 0, 3], dtype=int32)

In [55]:
ind_res_merged = central_toronto
ind_res_merged.columns= ['Postcode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']
# add clustering labels
ind_res_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
ind_res_merged = ind_res_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

ind_res_merged.head()# check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Thai Restaurant,Restaurant,Asian Restaurant,Italian Restaurant,Diner,Breakfast Spot,Jazz Club,Churrascaria,Japanese Restaurant,Office
1,M4P,Central Toronto,Davisville North,43.712751,-79.390197,2,Restaurant,Thai Restaurant,Asian Restaurant,Italian Restaurant,Sushi Restaurant,Steakhouse,Diner,Breakfast Spot,Indian Restaurant,Chinese Restaurant
2,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,1,Restaurant,Italian Restaurant,Thai Restaurant,Asian Restaurant,Breakfast Spot,Diner,Steakhouse,Sandwich Place,Japanese Restaurant,Jazz Club
3,M4S,Central Toronto,Davisville,43.704324,-79.388790,2,Restaurant,Thai Restaurant,Italian Restaurant,Indian Restaurant,Sushi Restaurant,Steakhouse,Japanese Restaurant,Asian Restaurant,Diner,Breakfast Spot
4,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,0,Italian Restaurant,Restaurant,Indian Restaurant,Sandwich Place,Chinese Restaurant,Filipino Restaurant,Breakfast Spot,Japanese Restaurant,Lounge,Mediterranean Restaurant


In [56]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
colors_array = cm.rainbow(np.linspace(0, 1, kclusters))
rainbow = [colors.rgb2hex(i) for i in colors_array]
print(rainbow)
# add markers to the map
markers_colors = []
for lat, lon, nei , cluster in zip(ind_res_merged['Latitude'], ind_res_merged['Longitude'], ind_res_merged['Neighborhood'], ind_res_merged['Cluster Labels']):
    label = folium.Popup(str(nei) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

['#8000ff', '#2adddd', '#d4dd80', '#ff0000']


## Cluster 1

In [57]:
cls1=ind_res_merged.loc[ind_res_merged['Cluster Labels'] == 0,ind_res_merged.columns[[2] + list(range(5, ind_res_merged.shape[1]))]]
cls1

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,"Moore Park, Summerhill East",0,Italian Restaurant,Restaurant,Indian Restaurant,Sandwich Place,Chinese Restaurant,Filipino Restaurant,Breakfast Spot,Japanese Restaurant,Lounge,Mediterranean Restaurant
6,Roselawn,0,Asian Restaurant,Restaurant,Italian Restaurant,Thai Restaurant,Diner,Steakhouse,Sandwich Place,Breakfast Spot,Eastern European Restaurant,General Entertainment
7,"Forest Hill North, Forest Hill West",0,Restaurant,Thai Restaurant,Asian Restaurant,Italian Restaurant,Diner,Japanese Restaurant,Chinese Restaurant,Steakhouse,General Entertainment,French Restaurant


## Cluster 2

In [58]:
cls2=ind_res_merged.loc[ind_res_merged['Cluster Labels'] == 1,ind_res_merged.columns[[2] + list(range(5, ind_res_merged.shape[1]))]]
cls2

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,North Toronto West,1,Restaurant,Italian Restaurant,Thai Restaurant,Asian Restaurant,Breakfast Spot,Diner,Steakhouse,Sandwich Place,Japanese Restaurant,Jazz Club
5,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",1,Restaurant,Italian Restaurant,Chinese Restaurant,Indian Restaurant,Sandwich Place,Japanese Restaurant,Hotel,Cantonese Restaurant,Café,Bistro


## Cluster 3

In [59]:
cls3=ind_res_merged.loc[ind_res_merged['Cluster Labels'] == 2,ind_res_merged.columns[[2] + list(range(5, ind_res_merged.shape[1]))]]
cls3

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Lawrence Park,2,Thai Restaurant,Restaurant,Asian Restaurant,Italian Restaurant,Diner,Breakfast Spot,Jazz Club,Churrascaria,Japanese Restaurant,Office
1,Davisville North,2,Restaurant,Thai Restaurant,Asian Restaurant,Italian Restaurant,Sushi Restaurant,Steakhouse,Diner,Breakfast Spot,Indian Restaurant,Chinese Restaurant
3,Davisville,2,Restaurant,Thai Restaurant,Italian Restaurant,Indian Restaurant,Sushi Restaurant,Steakhouse,Japanese Restaurant,Asian Restaurant,Diner,Breakfast Spot


## Cluster 4

In [60]:
cls4=ind_res_merged.loc[ind_res_merged['Cluster Labels'] == 3,ind_res_merged.columns[[2] + list(range(5, ind_res_merged.shape[1]))]]
cls4

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,"The Annex, North Midtown, Yorkville",3,Korean Restaurant,Chinese Restaurant,Restaurant,Asian Restaurant,Italian Restaurant,Nightclub,Vietnamese Restaurant,Middle Eastern Restaurant,Café,Caribbean Restaurant
